# Amazon Bedrock Prompt Caching and Routing Workshop

## Overview

This notebook demonstrates Amazon Bedrock's prompt caching and routing capabilities using the latest Claude models. You'll learn how to reduce latency and costs through intelligent prompt caching and how to route requests to optimal models based on your specific needs.

**Key Learning Outcomes:**
- Implement prompt caching to reduce costs and latency
- Use prompt routing for intelligent model selection
- Understand best practices for Bedrock API integration
- Monitor performance and usage statistics

## Context or Details about feature/use case

### Prompt Caching
Prompt caching allows you to cache frequently used prompts, reducing both latency and costs for subsequent requests. This is particularly useful for:
- Document analysis workflows
- Multi-turn conversations
- Repetitive query patterns

### Prompt Routing
Prompt routing intelligently directs requests to the most appropriate model based on:
- Query complexity
- Cost optimization requirements
- Performance needs
- Model capabilities

### Supported Models
- **Claude Haiku 4.5**: Fast, cost-effective for simple tasks
- **Claude Sonnet 4.5**: Balanced performance and cost
- **Claude Opus 4.1**: Most capable for complex reasoning
- **Amazon Nova Models**: Latest AWS-native models

## Prerequisites

Before running this notebook, ensure you have:

1. **AWS Account** with appropriate permissions
2. **Amazon Bedrock access** with Claude models enabled
3. **AWS CLI configured** with credentials
4. **Python 3.8+** installed
5. **Required Python packages** (installed in Setup section)

### Required AWS Permissions
Your AWS credentials need the following permissions:
- `bedrock:InvokeModel`
- `bedrock:ListFoundationModels`
- `bedrock:GetModelInvocationLoggingConfiguration`

## Setup

Let's install the required dependencies and set up our environment.

In [ ]:
# Install required packages
!pip install boto3 streamlit pandas numpy

In [ ]:
# Import required libraries
import boto3
import json
import time
from datetime import datetime
import pandas as pd
from typing import Dict, List, Optional, Tuple

# Initialize Bedrock client
bedrock_client = boto3.client('bedrock-runtime', region_name='us-east-1')

print("✅ Setup complete! Bedrock client initialized.")

## Your code with comments starts here

### Model Manager Class

First, let's create a model manager to handle different Claude models and their configurations.

In [ ]:
class ModelManager:
    """Manages Bedrock model selection and configuration"""
    
    def __init__(self):
        self.models = {
            'haiku': 'anthropic.claude-3-haiku-20240307-v1:0',
            'sonnet': 'anthropic.claude-3-5-sonnet-20241022-v2:0',
            'opus': 'anthropic.claude-3-opus-20240229-v1:0'
        }
        
    def get_model_id(self, model_name: str) -> str:
        """Get the full model ID for a given model name"""
        return self.models.get(model_name.lower(), self.models['sonnet'])
    
    def list_available_models(self) -> List[str]:
        """List all available model names"""
        return list(self.models.keys())

# Initialize model manager
model_manager = ModelManager()
print(f"Available models: {model_manager.list_available_models()}")

### Bedrock Service Class

Now let's create a service class to handle Bedrock API interactions with caching capabilities.

In [ ]:
class BedrockService:
    """Service class for Bedrock API interactions with caching"""
    
    def __init__(self, client):
        self.client = client
        self.cache = {}  # Simple in-memory cache
        self.cache_stats = {'hits': 0, 'misses': 0}
    
    def _generate_cache_key(self, model_id: str, prompt: str) -> str:
        """Generate a cache key for the prompt"""
        return f"{model_id}:{hash(prompt)}"
    
    def invoke_model_with_cache(self, model_id: str, prompt: str, use_cache: bool = True) -> Dict:
        """Invoke model with optional caching"""
        cache_key = self._generate_cache_key(model_id, prompt)
        
        # Check cache first
        if use_cache and cache_key in self.cache:
            self.cache_stats['hits'] += 1
            print("🎯 Cache HIT - Using cached response")
            return self.cache[cache_key]
        
        # Cache miss - make API call
        self.cache_stats['misses'] += 1
        print("🔄 Cache MISS - Making API call")
        
        start_time = time.time()
        
        # Prepare request body
        body = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 1000,
            "messages": [
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        }
        
        # Make API call
        response = self.client.invoke_model(
            modelId=model_id,
            body=json.dumps(body)
        )
        
        # Parse response
        response_body = json.loads(response['body'].read())
        
        # Add timing information
        response_body['latency_ms'] = round((time.time() - start_time) * 1000, 2)
        response_body['timestamp'] = datetime.now().isoformat()
        
        # Cache the response
        if use_cache:
            self.cache[cache_key] = response_body
        
        return response_body
    
    def get_cache_stats(self) -> Dict:
        """Get cache performance statistics"""
        total = self.cache_stats['hits'] + self.cache_stats['misses']
        hit_rate = (self.cache_stats['hits'] / total * 100) if total > 0 else 0
        
        return {
            'cache_hits': self.cache_stats['hits'],
            'cache_misses': self.cache_stats['misses'],
            'hit_rate_percent': round(hit_rate, 2),
            'cached_items': len(self.cache)
        }

# Initialize Bedrock service
bedrock_service = BedrockService(bedrock_client)
print("✅ Bedrock service initialized with caching capabilities")

### Prompt Router Class

Let's create a prompt router that intelligently selects the best model based on query characteristics.

In [ ]:
class PromptRouter:
    """Intelligent prompt routing based on query characteristics"""
    
    def __init__(self, model_manager: ModelManager):
        self.model_manager = model_manager
        self.routing_stats = {}
    
    def analyze_query_complexity(self, prompt: str) -> str:
        """Analyze prompt complexity and return complexity level"""
        word_count = len(prompt.split())
        
        # Simple heuristics for complexity
        complex_keywords = ['analyze', 'compare', 'evaluate', 'reasoning', 'complex', 'detailed']
        simple_keywords = ['summarize', 'list', 'what is', 'define', 'simple']
        
        has_complex = any(keyword in prompt.lower() for keyword in complex_keywords)
        has_simple = any(keyword in prompt.lower() for keyword in simple_keywords)
        
        if word_count > 100 or has_complex:
            return 'complex'
        elif word_count < 20 or has_simple:
            return 'simple'
        else:
            return 'medium'
    
    def route_prompt(self, prompt: str, priority: str = 'balanced') -> Tuple[str, str]:
        """Route prompt to optimal model based on complexity and priority"""
        complexity = self.analyze_query_complexity(prompt)
        
        # Routing logic
        if priority == 'cost':
            model_name = 'haiku'  # Always use cheapest
        elif priority == 'performance':
            model_name = 'opus'   # Always use most capable
        else:  # balanced
            if complexity == 'simple':
                model_name = 'haiku'
            elif complexity == 'complex':
                model_name = 'opus'
            else:
                model_name = 'sonnet'
        
        # Track routing decisions
        if model_name not in self.routing_stats:
            self.routing_stats[model_name] = 0
        self.routing_stats[model_name] += 1
        
        model_id = self.model_manager.get_model_id(model_name)
        
        print(f"🎯 Routing Decision: {complexity} complexity → {model_name} model")
        
        return model_id, model_name
    
    def get_routing_stats(self) -> Dict:
        """Get routing statistics"""
        return self.routing_stats.copy()

# Initialize prompt router
prompt_router = PromptRouter(model_manager)
print("✅ Prompt router initialized")

### Demo 1: Prompt Caching in Action

Let's demonstrate how prompt caching works by making repeated requests.

In [ ]:
# Sample document for caching demo
sample_document = """
Amazon Web Services (AWS) is a comprehensive cloud computing platform provided by Amazon. 
It offers over 200 fully featured services from data centers globally. AWS serves millions 
of customers including startups, large enterprises, and government agencies. Key services 
include compute power, database storage, content delivery, and machine learning capabilities.
"""

# First request - will be cached
prompt1 = f"Based on this document: {sample_document}\n\nQuestion: What is AWS?"
model_id = model_manager.get_model_id('sonnet')

print("=== First Request (Cache Miss Expected) ===")
response1 = bedrock_service.invoke_model_with_cache(model_id, prompt1, use_cache=True)
print(f"Response: {response1['content'][0]['text'][:100]}...")
print(f"Latency: {response1['latency_ms']}ms")
print()

# Second identical request - should hit cache
print("=== Second Identical Request (Cache Hit Expected) ===")
response2 = bedrock_service.invoke_model_with_cache(model_id, prompt1, use_cache=True)
print(f"Response: {response2['content'][0]['text'][:100]}...")
print(f"Latency: {response2['latency_ms']}ms")
print()

# Display cache statistics
cache_stats = bedrock_service.get_cache_stats()
print("=== Cache Performance ===")
for key, value in cache_stats.items():
    print(f"{key}: {value}")

### Demo 2: Intelligent Prompt Routing

Now let's see how the prompt router selects different models based on query complexity.

In [ ]:
# Test different types of queries
test_queries = [
    "What is machine learning?",  # Simple
    "Explain the differences between supervised and unsupervised learning algorithms, including their use cases and performance characteristics.",  # Complex
    "List the main AWS compute services.",  # Simple
    "Analyze the trade-offs between microservices and monolithic architectures in cloud-native applications."  # Complex
]

print("=== Prompt Routing Demonstration ===")
results = []

for i, query in enumerate(test_queries, 1):
    print(f"\n--- Query {i} ---")
    print(f"Query: {query[:60]}...")
    
    # Route the prompt
    model_id, model_name = prompt_router.route_prompt(query, priority='balanced')
    
    # Make the request (without caching for this demo)
    response = bedrock_service.invoke_model_with_cache(model_id, query, use_cache=False)
    
    results.append({
        'query': query[:50] + '...',
        'model': model_name,
        'latency_ms': response['latency_ms'],
        'response_length': len(response['content'][0]['text'])
    })
    
    print(f"Selected Model: {model_name}")
    print(f"Latency: {response['latency_ms']}ms")
    print(f"Response: {response['content'][0]['text'][:100]}...")

# Display routing statistics
print("\n=== Routing Statistics ===")
routing_stats = prompt_router.get_routing_stats()
for model, count in routing_stats.items():
    print(f"{model}: {count} requests")

### Demo 3: Performance Comparison

Let's compare performance with and without caching, and across different routing strategies.

In [ ]:
# Performance comparison
test_prompt = "Explain the benefits of cloud computing for small businesses."
model_id = model_manager.get_model_id('sonnet')

# Test without caching
print("=== Performance Comparison ===")
print("\n1. Without Caching:")
times_no_cache = []
for i in range(3):
    response = bedrock_service.invoke_model_with_cache(model_id, test_prompt, use_cache=False)
    times_no_cache.append(response['latency_ms'])
    print(f"  Request {i+1}: {response['latency_ms']}ms")

# Test with caching
print("\n2. With Caching:")
times_with_cache = []
for i in range(3):
    response = bedrock_service.invoke_model_with_cache(model_id, test_prompt, use_cache=True)
    times_with_cache.append(response['latency_ms'])
    print(f"  Request {i+1}: {response['latency_ms']}ms")

# Calculate savings
avg_no_cache = sum(times_no_cache) / len(times_no_cache)
avg_with_cache = sum(times_with_cache) / len(times_with_cache)
savings_percent = ((avg_no_cache - avg_with_cache) / avg_no_cache) * 100

print(f"\n=== Performance Summary ===")
print(f"Average latency without cache: {avg_no_cache:.2f}ms")
print(f"Average latency with cache: {avg_with_cache:.2f}ms")
print(f"Performance improvement: {savings_percent:.1f}%")

# Final cache statistics
final_stats = bedrock_service.get_cache_stats()
print(f"\n=== Final Cache Statistics ===")
for key, value in final_stats.items():
    print(f"{key}: {value}")

## Other Considerations or Advanced section or Best Practices

### Best Practices for Prompt Caching

1. **Cache Key Design**: Use meaningful cache keys that include model version and relevant parameters
2. **TTL Management**: Implement time-to-live (TTL) for cache entries to ensure freshness
3. **Memory Management**: Monitor cache size and implement eviction policies for production use
4. **Cache Warming**: Pre-populate cache with frequently used prompts

### Best Practices for Prompt Routing

1. **Complexity Analysis**: Develop sophisticated heuristics for query complexity
2. **Cost Monitoring**: Track costs across different models to optimize routing decisions
3. **Performance Metrics**: Monitor latency and quality metrics for each model
4. **Fallback Strategies**: Implement fallback models for high availability

### Production Considerations

- **Persistent Caching**: Use Redis or DynamoDB for distributed caching
- **Monitoring**: Implement comprehensive logging and metrics
- **Security**: Ensure sensitive data is not cached inappropriately
- **Rate Limiting**: Implement proper rate limiting for API calls
- **Error Handling**: Add robust error handling and retry logic

In [ ]:
# Example of production-ready cache with TTL
import time
from typing import Optional

class ProductionCache:
    """Production-ready cache with TTL and size limits"""
    
    def __init__(self, max_size: int = 1000, default_ttl: int = 3600):
        self.cache = {}
        self.timestamps = {}
        self.max_size = max_size
        self.default_ttl = default_ttl
    
    def get(self, key: str) -> Optional[dict]:
        """Get item from cache if not expired"""
        if key not in self.cache:
            return None
        
        # Check if expired
        if time.time() - self.timestamps[key] > self.default_ttl:
            del self.cache[key]
            del self.timestamps[key]
            return None
        
        return self.cache[key]
    
    def set(self, key: str, value: dict) -> None:
        """Set item in cache with size management"""
        # Evict oldest items if at capacity
        if len(self.cache) >= self.max_size:
            oldest_key = min(self.timestamps.keys(), key=lambda k: self.timestamps[k])
            del self.cache[oldest_key]
            del self.timestamps[oldest_key]
        
        self.cache[key] = value
        self.timestamps[key] = time.time()

# Example usage
prod_cache = ProductionCache(max_size=100, default_ttl=1800)  # 30 minutes TTL
print("✅ Production cache example created")

## Next Steps

Now that you've learned the basics of prompt caching and routing, here are some next steps to explore:

### 1. Advanced Routing Strategies
- Implement machine learning-based routing decisions
- Add user preference learning
- Develop cost-aware routing algorithms

### 2. Integration Patterns
- Build a REST API wrapper around these capabilities
- Create a Streamlit web application for interactive use
- Integrate with existing applications and workflows

### 3. Monitoring and Analytics
- Set up CloudWatch metrics for cache performance
- Implement cost tracking across different models
- Create dashboards for routing decision analysis

### 4. Scale and Production
- Deploy using AWS Lambda for serverless scaling
- Implement distributed caching with ElastiCache
- Add comprehensive error handling and logging

### 5. Explore Additional Features
- Multi-modal prompt routing (text, images, documents)
- Streaming responses with caching
- Custom model fine-tuning integration

## Cleanup

Let's clean up any resources and display final statistics.

In [ ]:
# Display final statistics
print("=== Workshop Summary ===")
print(f"Total API calls made: {bedrock_service.cache_stats['hits'] + bedrock_service.cache_stats['misses']}")
print(f"Cache hits: {bedrock_service.cache_stats['hits']}")
print(f"Cache misses: {bedrock_service.cache_stats['misses']}")
print(f"Cache hit rate: {bedrock_service.get_cache_stats()['hit_rate_percent']}%")
print(f"Items in cache: {len(bedrock_service.cache)}")

print("\n=== Model Usage ===")
routing_stats = prompt_router.get_routing_stats()
for model, count in routing_stats.items():
    print(f"{model}: {count} requests")

# Clear cache to free memory
bedrock_service.cache.clear()
bedrock_service.cache_stats = {'hits': 0, 'misses': 0}

print("\n✅ Cleanup complete! Cache cleared and statistics reset.")
print("\n🎉 Workshop completed successfully!")
print("\nYou've learned how to:")
print("- Implement prompt caching for cost and latency optimization")
print("- Use intelligent prompt routing for model selection")
print("- Monitor performance and usage statistics")
print("- Apply best practices for production deployments")